In [194]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import keras
from tqdm import tqdm

In [195]:
df = pd.read_csv('/home/hikkav/hack/data/landmarks/train.csv')

In [196]:
df.head()

,im_name,height,width,x0,y0,x1,y1,x2,y2,x3,...,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16
0,0000.jpg,1242,2208,876.0,491.0,888.0,525.0,883.0,461.0,908.0,...,981.5,473.0,875.0,569.0,888.0,552.0,953.0,579.0,888.0,575.0
1,0001.jpg,1242,2208,1068.0,558.0,1061.0,589.0,993.0,486.0,1041.0,...,1129.5,519.0,1000.0,620.0,1058.0,615.0,1101.0,636.0,1051.0,641.0
2,0002.jpg,1242,2208,1027.0,568.0,1029.0,607.0,966.0,545.0,1006.0,...,1089.5,552.0,986.0,662.0,1027.0,643.0,1073.0,665.0,1025.0,673.0
3,0003.jpg,1242,2208,1024.0,566.0,1025.0,606.0,964.0,546.0,1004.0,...,1084.0,551.0,983.0,660.0,1024.0,640.0,1071.0,664.0,1023.0,671.0
4,0004.jpg,1242,2208,970.0,486.0,979.0,534.0,909.0,466.0,961.0,...,1060.5,453.0,935.0,611.0,981.0,579.0,1047.0,602.0,983.0,621.0


In [197]:
abs_path = '/home/hikkav/hack/data/landmarks/train_images/'

In [198]:
path_to_save_img = '/home/hikkav/GlobalLogicNet/new_data/'

In [199]:
cfg = '/home/hikkav/PycharmProjects/makeProjectData/cfg/yolov3-face.cfg'

In [200]:
weights = '/home/hikkav/PycharmProjects/makeProjectData/weights/yolov3-wider_16000.weights'

In [201]:
df['im_name'] = df['im_name'].apply(lambda x: abs_path+x)

In [202]:
net = cv2.dnn.readNetFromDarknet(cfg, weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [203]:
def unlist_lands(landmarks):
    lands = []
    for i in landmarks:
        lands.append(i[0])
        lands.append(i[1])
    return lands

In [204]:
def make_new_data(df):
    list_ = []
    for i in tqdm(df.iterrows()):
        img = Image.open(i[1][0])
        img, landmarks = crop(img, i[1][3:])
        cv2.imwrite(path_to_save_img+i[1][0].split('/')[-1],img)
        landmarks = unlist_lands(landmarks)
        list_.append((path_to_save_img+i[1][0].split('/')[-1],i[1][1], i[1][2], *landmarks))
    names = ['im_name','height', 'width']+list(df.columns[3:])
    print(names)
    new_df = pd.DataFrame(data=list_, columns=names)
    
    return new_df

In [205]:
def resize_image_landmarks(image, new_height, new_width, landmarks):
    cur_height = image.height
    cur_width = image.width
    image = image.resize((new_height, new_width))
    for i in range(len(landmarks)):
        landmarks[i] = (
            new_width/ cur_width * landmarks[i][0],
            new_height / cur_height * landmarks[i][1]
            )
    return image, landmarks

In [206]:
def crop(image, landmarks):
    param = 60
    blob = cv2.dnn.blobFromImage(np.array(image), 1 / 255, (416, 416),
                                             [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    outs = net.forward(get_outputs_names(net))
    faces, _, __ = post_process(np.array(image), outs, 0.7, 0.8)
    crop_faces = [(faces[i][0]-param, faces[i][1]-param, faces[i][2] + faces[i][0]+param, faces[i][3] + faces[i][1]+param) for i in
                  range(len(faces))]
    landmarks = list(zip(
                landmarks[[i for i in range(0,34,1) if i%2==0]],landmarks[[i  for i in range(0,34,1) if i%2!=0] ]))
    landmarks = [(i[0]-crop_faces[0][0], i[1]-crop_faces[0][1]) for i in landmarks]
    img = image.crop(crop_faces[0])
    img , landmarks = resize_image_landmarks(img, 128,128, landmarks)
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return np.array(img), landmarks

    

In [207]:
def get_outputs_names(net):
        layers_names = net.getLayerNames()
        return [layers_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [208]:
def post_process(frame, outs, conf_threshold, nms_threshold):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.
    confidences = []
    boxes = []

    people_boxes = []
    center = []
    class_ids = []
    if True:

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > conf_threshold:
                    center_x = int(detection[0] * frame_width)
                    center_y = int(detection[1] * frame_height)
                    width = int(detection[2] * frame_width)
                    height = int(detection[3] * frame_height)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])
                    center.append([center_x, center_y])
                    class_ids.append(class_id)

    # Perform non maximum suppression to eliminate redundant
    # overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold,
                               nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]

        people_boxes.append(box)

    return people_boxes, class_ids, indices

In [209]:
new_df = make_new_data(df)


0it [00:00, ?it/s]
1it [00:00,  2.53it/s]
2it [00:00,  2.78it/s]
3it [00:00,  3.00it/s]
4it [00:01,  3.15it/s]
5it [00:01,  3.27it/s]
6it [00:01,  3.35it/s]
7it [00:02,  3.36it/s]
8it [00:02,  3.39it/s]
9it [00:02,  3.47it/s]
10it [00:02,  3.50it/s]
11it [00:03,  3.47it/s]
12it [00:03,  3.37it/s]
13it [00:03,  3.15it/s]
14it [00:04,  3.09it/s]
15it [00:04,  3.17it/s]
16it [00:04,  3.26it/s]
17it [00:05,  3.24it/s]
18it [00:05,  3.30it/s]
19it [00:05,  3.34it/s]
20it [00:06,  3.37it/s]
21it [00:06,  3.39it/s]
22it [00:06,  3.32it/s]
23it [00:06,  3.32it/s]
24it [00:07,  3.34it/s]
25it [00:07,  3.29it/s]
26it [00:07,  3.25it/s]
27it [00:08,  3.14it/s]
28it [00:08,  3.20it/s]
29it [00:08,  3.26it/s]
30it [00:09,  3.29it/s]
31it [00:09,  3.33it/s]
32it [00:09,  3.33it/s]
33it [00:09,  3.36it/s]
34it [00:10,  3.37it/s]
35it [00:10,  3.38it/s]
36it [00:10,  3.44it/s]
37it [00:11,  3.44it/s]
38it [00:11,  3.46it/s]
39it [00:11,  3.47it/s]
40it [00:11,  3.48it/s]
41it [00:12,  3.45it/s]
42it 

['im_name', 'height', 'width', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13', 'y13', 'x14', 'y14', 'x15', 'y15', 'x16', 'y16']


In [193]:
new_df.to_csv('new_data.csv',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'presentation/new_data.csv'